In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
agents_df = session.sql("SHOW AGENTS IN SCHEMA snowflake_intelligence.agents").to_pandas()
agents_df

In [ ]:
def fetch_semantic_model_paths(session):
    results = []
    agent_names = agents_df[1].tolist()
    
    for agent in agent_names:
        describe_sql = f'DESCRIBE AGENT SNOWFLAKE_INTELLIGENCE.AGENTS."{agent}"'
        df_desc = session.sql(describe_sql).to_pandas()
         # Each DESCRIBE returns an 'AGENT_SPEC' column as a JSON string
        
        agent_spec_json = df_desc[6].iloc[0]
            
        if agent_spec_json:
            try:
                import json
                spec = json.loads(agent_spec_json)
                
                if 'tool_resources' in spec:
                    for tool_name, tool_data in spec['tool_resources'].items():
                        semantic_file = tool_data.get('semantic_model_file')
                        results.append({
                            "agent_name": agent,
                            "tool_name": tool_name,
                            "semantic_model_file": semantic_file
                        })
            except json.JSONDecodeError:
                pass
        
    # Convert to DataFrame for display
    df_results = pd.DataFrame(results)
    return df_results

def get_cortex_analyst_logs(SEMANTIC_MODEL_FILE):
    cortex_analyst_log = session.sql(f'''SELECT 
          timestamp,
          request_id,
          semantic_model_name,
          tables_referenced,
          user_name,
          source,
          feedback,
          response_status_code,
          request_body:messages[0].content[0].text::STRING as user_question,
          response_body:response_metadata.analyst_latency_ms::NUMBER as latency_ms,
          generated_sql,
          response_body:response_metadata.analyst_orchestration_path::STRING as orchestration_path,
          response_body:response_metadata.question_category::STRING as question_category,
          response_body:message.content[1].confidence.verified_query_used.name::STRING as verified_query_name,
         response_body:message.content[1].confidence.verified_query_used.question::STRING as verified_query_question
        FROM TABLE(
          SNOWFLAKE.LOCAL.CORTEX_ANALYST_REQUESTS('FILE_ON_STAGE', '{SEMANTIC_MODEL_FILE}'))''')
    
    df = cortex_analyst_log.to_pandas()
    df["QUERY_TYPE"] = df["ORCHESTRATION_PATH"].apply(
    lambda x: "Verified Query" if x == "vqr_fast_path" else "Non-Verified Query"
)   
    df['CORTEX_ANALYST_CREDITS'] = 67/1000
    
    # Push back into Snowflake (append to table)
    session.write_pandas(
        df,
        table_name="CORTEX_ANALYST_LOGS",
        database=None,   # or specify your DB
        schema=None,     # or specify your schema
        auto_create_table=False,  # you already created the table
        overwrite=False
    )
    return df
    
def verified_query_count(df):
    # Breakdown by semantic model
    semantic_model_summary = (
        df.groupby(['SEMANTIC_MODEL_NAME', 'QUERY_TYPE'])
        .size()
        .reset_index(name="request_count")
    )
    
    # Calculate percentages within each semantic model
    semantic_model_summary["percentage"] = (
        semantic_model_summary
        .groupby('SEMANTIC_MODEL_NAME')['request_count']
        .transform(lambda x: round((x * 100.0) / x.sum(), 2))
    )
    
    # Sort by semantic model and request count
    semantic_model_summary = semantic_model_summary.sort_values(
        ['SEMANTIC_MODEL_NAME', 'request_count'], 
        ascending=[True, False]
    ).reset_index(drop=True)
    
    return semantic_model_summary

def top_verified_queries(df):
    # Breakdown by semantic model
    semantic_model_top = (
        df[df['QUERY_TYPE'] == 'Verified Query']
        .groupby(["SEMANTIC_MODEL_NAME", "VERIFIED_QUERY_NAME", "VERIFIED_QUERY_QUESTION"])
        .size()
        .reset_index(name="frequency")
    )
    
    # Calculate percentages within each semantic model
    semantic_model_top["percentage_of_verified_queries"] = (
        semantic_model_top
        .groupby('SEMANTIC_MODEL_NAME')['frequency']
        .transform(lambda x: round((x * 100.0) / x.sum(), 2))
    )
    
    # Sort by semantic model and frequency, take top 10 per model
    semantic_model_top = (
        semantic_model_top
        .sort_values(['SEMANTIC_MODEL_NAME', 'frequency'], ascending=[True, False])
        .groupby('SEMANTIC_MODEL_NAME')
        .head(10)
        .reset_index(drop=True)
    )

    return semantic_model_top

def slowest_queries(df, number=10):
    # Filter out rows where latency_ms is null
    slow_queries = df[df["LATENCY_MS"].notnull()].copy()
    
    # Compute latency in seconds
    slow_queries["LATENCY_SECONDS"] = (slow_queries["LATENCY_MS"] / 1000.0).round(2)
    
    # Breakdown by semantic model - top slowest per model
    semantic_model_slow = slow_queries[[
        "SEMANTIC_MODEL_NAME",
        "USER_QUESTION",
        "LATENCY_SECONDS",
        "ORCHESTRATION_PATH",
        "QUESTION_CATEGORY"
    ]].sort_values(['SEMANTIC_MODEL_NAME', 'LATENCY_SECONDS'], ascending=[True, False])
    
    # Take top N slowest per semantic model
    semantic_model_slow = (
        semantic_model_slow
        .groupby('SEMANTIC_MODEL_NAME')
        .head(number)
        .reset_index(drop=True)
    )
    
    return semantic_model_slow

def latency_summary_by_semantic_model(df):
    """
    Provides latency statistics breakdown by semantic model.
    """
    # Filter out rows where latency_ms is null
    latency_data = df[df["LATENCY_MS"].notnull()].copy()
    
    # Compute latency in seconds
    latency_data["LATENCY_SECONDS"] = (latency_data["LATENCY_MS"] / 1000.0).round(2)
    
    # Stats by semantic model
    semantic_model_stats = (
        latency_data
        .groupby('SEMANTIC_MODEL_NAME')['LATENCY_SECONDS']
        .agg(['count', 'mean', 'median', 'min', 'max'])
        .round(2)
        .reset_index()
    )
    
    semantic_model_stats.columns = [
        'SEMANTIC_MODEL_NAME', 'query_count', 'avg_latency_seconds', 
        'median_latency_seconds', 'min_latency_seconds', 'max_latency_seconds'
    ]
    
    return semantic_model_stats

def create_sf_intelligence_query_history(session, target_table="snowflake_intelligence.public.sf_intelligence_query_history"):
    """
    Creates or replaces a table with Cortex Agent query history joined to query attribution.
    Keeps last 30 days of queries with compute credits > 0 and cleans query text for matching.
    
    Parameters
    ----------
    session : snowflake.snowpark.Session
        Active Snowpark session.
    target_table : str, optional
        Fully qualified name of the target table.
    """
    
    query = f"""
    CREATE OR REPLACE TABLE {target_table} AS
    SELECT 
        qh.query_id,
        qh.query_text,
        qh.start_time,
        qh.total_elapsed_time,
        qh.warehouse_name,
        qh.user_name,
        qah.credits_attributed_compute,
        -- Clean the generated SQL for matching
        TRIM(REGEXP_REPLACE(
            REGEXP_REPLACE(
                REGEXP_REPLACE(qh.query_text, '--[^\\n]*\\n', '\\n'),
                '/\\*.*?\\*/', ' '
            ),
            '\\s+', ' '
        )) AS cleaned_query_text
    FROM SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY qh
    JOIN SNOWFLAKE.ACCOUNT_USAGE.QUERY_ATTRIBUTION_HISTORY qah 
        ON qh.query_id = qah.query_id
    WHERE qh.query_tag = 'cortex-agent'
      AND qh.start_time >= DATEADD(DAY, -30, CURRENT_TIMESTAMP())
      AND qah.credits_attributed_compute > 0
    """
    
    session.sql(query).collect()
    print(f"✅ Table {target_table} created/updated successfully.")

    

In [ ]:
df_results = fetch_semantic_model_paths(session)
df_results

In [ ]:
CREATE OR REPLACE TABLE CORTEX_ANALYST_LOGS (
    TIMESTAMP                TIMESTAMP_NTZ,
    REQUEST_ID               STRING,
    SEMANTIC_MODEL_NAME      STRING,
    TABLES_REFERENCED        STRING,
    USER_NAME                STRING,
    SOURCE                   STRING,
    FEEDBACK                 STRING,
    RESPONSE_STATUS_CODE     INTEGER,
    USER_QUESTION            STRING,
    LATENCY_MS               NUMBER,
    GENERATED_SQL            STRING,
    ORCHESTRATION_PATH       STRING,
    QUESTION_CATEGORY        STRING,
    VERIFIED_QUERY_NAME      STRING,
    VERIFIED_QUERY_QUESTION  STRING,
    QUERY_TYPE               STRING,
    CORTEX_ANALYST_CREDITS   FLOAT
);


In [ ]:
test_2 = latency_summary_by_semantic_model(df_agg)
test_2

In [ ]:
for file in df_results['semantic_model_file']:
    if file != None:
        df = get_cortex_analyst_logs(file)

In [ ]:
select * from CORTEX_ANALYST_LOGS limit 100;

In [ ]:
# SEMANTIC_MODEL_FILE = '@E2E_SNOW_MLOPS_DB.STREAMLIT.SEMANTIC/mortgage_lending_prediction_v2.yaml'

# df = get_cortex_analyst_logs(SEMANTIC_MODEL_FILE)

# vq_sum_df = verified_query_count(df)
# top_verified_queries_df = top_verified_queries(df)
# slowest_queries_df = slowest_queries(df)
# df

In [ ]:
print(df.columns)

In [ ]:
import streamlit as st

# Verified vs Non-Verified Queries
st.subheader("Verified vs Non-Verified Queries")
st.dataframe(vq_sum_df)
st.bar_chart(vq_sum_df.set_index("QUERY_TYPE")["request_count"])

# Top 10 Verified Queries
st.subheader("Top 10 Verified Queries")
st.dataframe(top_verified_queries_df)
st.bar_chart(top_verified_queries_df.set_index("VERIFIED_QUERY_QUESTION")["frequency"])

# Slowest Queries
st.subheader("Top 10 Slowest Queries")
st.dataframe(slowest_queries_df)
st.bar_chart(slowest_queries_df.set_index("USER_QUESTION")["LATENCY_SECONDS"])


In [ ]:
##THIS IS SLOW - ONLY RUN THIS IF YOU HAVEN'T ALREADY
# create_sf_intelligence_query_history(session)

In [ ]:

sfi_query_history = session.table("snowflake_intelligence.public.sf_intelligence_query_history")
pd_sfi_query_history = sfi_query_history.to_pandas()
pd_sfi_query_history.set_index("CLEANED_QUERY_TEXT")
df.set_index("GENERATED_SQL")

##joining on generated SQL
ca_query_history = pd.merge(pd_sfi_query_history, df, left_index=True, right_index=True)
ca_query_history['TOTAL_TIME'] = ca_query_history["TOTAL_ELAPSED_TIME"] + ca_query_history["LATENCY_MS"]
ca_query_history['TOTAL_CREDITS_WH_AND_CA'] = ca_query_history["CREDITS_ATTRIBUTED_COMPUTE"] + ca_query_history["CORTEX_ANALYST_CREDITS"]

In [ ]:
total_credits_attributed_compute = round(ca_query_history["CREDITS_ATTRIBUTED_COMPUTE"].sum(), 4)
st.metric(label="Total Attributed WH Query Credits", value=total_credits_attributed_compute)

# Cost analysis
total_cost_by_user = ca_query_history.groupby('USER_NAME_y')['CREDITS_ATTRIBUTED_COMPUTE'].sum()
st.dataframe(total_cost_by_user)

total_cortex_analyst_credits_by_user = ca_query_history.groupby('USER_NAME_y')['TOTAL_CREDITS_WH_AND_CA'].sum()
st.dataframe(total_cortex_analyst_credits_by_user)

In [ ]:
print(ca_query_history.columns)

In [ ]:
avg_total_latency_ms = round(ca_query_history['TOTAL_TIME'].mean())
st.metric(label="Average Total Latency (CA + SQL Execution) ms", value=avg_total_latency_ms)
ca_latency_stats = ca_query_history['LATENCY_MS'].describe()
st.subheader("Cortex Analyst Latency Stats (MS)")
st.dataframe(ca_latency_stats)

wh_query_latency_stats = ca_query_history['TOTAL_ELAPSED_TIME'].describe()
st.subheader("SQL Execution WH Latency Stats (MS)")
st.dataframe(wh_query_latency_stats)

In [ ]:
# User adoption and activity
user_activity = ca_query_history.groupby('USER_NAME_y').agg({
    'QUERY_ID': 'count',
    'LATENCY_MS': 'mean',
    'CREDITS_ATTRIBUTED_COMPUTE': 'sum'
}).rename(columns={'QUERY_ID': 'total_queries'})
st.subheader("User Activity")
st.dataframe(user_activity)

# Query frequency over time
ca_query_history['date'] = pd.to_datetime(ca_query_history['TIMESTAMP']).dt.date
daily_usage = ca_query_history.groupby('date')['QUERY_ID'].count()


# Most used semantic models
model_usage = ca_query_history['SEMANTIC_MODEL_NAME'].value_counts()

In [ ]:
SELECT sum FROM SNOWFLAKE.account_usage.CORTEX_ANALYST_USAGE_HISTORY

In [ ]:
def user_activity_by_semantic_model(df):
    """
    Provides user activity analysis by semantic model.
    Returns a DataFrame with user activity broken down by semantic model.
    """
    import pandas as pd
    
    # User activity by semantic model
    user_model_activity = df.groupby(['USER_NAME', 'SEMANTIC_MODEL_NAME']).agg({
        'REQUEST_ID': 'count',  # Adjust column name if different
        'LATENCY_MS': 'mean',
        'CORTEX_ANALYST_CREDITS': 'sum'
    }).rename(columns={
        'REQUEST_ID': 'queries_count',
        'LATENCY_MS': 'avg_latency_ms',
        'CORTEX_ANALYST_CREDITS': 'total_credits'
    }).round(2).reset_index()
    
    # Add percentage of user's queries per model
    user_totals = user_model_activity.groupby('USER_NAME')['queries_count'].sum()
    user_model_activity['percentage_of_user_queries'] = (
        user_model_activity.apply(
            lambda row: round((row['queries_count'] / user_totals[row['USER_NAME']]) * 100, 2),
            axis=1
        )
    )
    
    # Sort by user name and query count
    user_model_activity = user_model_activity.sort_values(
        ['USER_NAME', 'queries_count'], 
        ascending=[True, False]
    ).reset_index(drop=True)
    
    return user_model_activity


def user_activity_by_semantic_model(df):
    """
    Provides user activity analysis by semantic model.
    Returns a DataFrame with user activity broken down by semantic model.
    """
    import pandas as pd
    
    # User activity by semantic model
    user_model_activity = df.groupby(['USER_NAME', 'SEMANTIC_MODEL_NAME']).agg({
        'REQUEST_ID': 'count',  # Adjust column name if different
        'LATENCY_MS': 'mean',
        'CORTEX_ANALYST_CREDITS': 'sum'
    }).rename(columns={
        'REQUEST_ID': 'queries_count',
        'LATENCY_MS': 'avg_latency_ms',
        'CORTEX_ANALYST_CREDITS': 'total_credits'
    }).round(2).reset_index()
    
    # Add percentage of user's queries per model
    user_totals = user_model_activity.groupby('USER_NAME')['queries_count'].sum()
    user_model_activity['percentage_of_user_queries'] = (
        user_model_activity.apply(
            lambda row: round((row['queries_count'] / user_totals[row['USER_NAME']]) * 100, 2),
            axis=1
        )
    )
    
    # Sort by user name and query count
    user_model_activity = user_model_activity.sort_values(
        ['USER_NAME', 'queries_count'], 
        ascending=[True, False]
    ).reset_index(drop=True)
    
    return user_model_activity


def semantic_model_usage_summary(df):
    """
    Provides overall semantic model usage summary.
    Returns a DataFrame with usage statistics per semantic model.
    """
    model_usage = df.groupby('SEMANTIC_MODEL_NAME').agg({
        'REQUEST_ID': 'count',  # Adjust column name if different
        'USER_NAME': 'nunique',
        'LATENCY_MS': 'mean',
        'CORTEX_ANALYST_CREDITS': 'sum'
    }).rename(columns={
        'REQUEST_ID': 'total_queries',
        'USER_NAME': 'unique_users',
        'LATENCY_MS': 'avg_latency_ms',
        'CORTEX_ANALYST_CREDITS': 'total_cortex_analyst_credits'
    }).round(2).reset_index()
    
    # Add percentage of total queries
    model_usage['percentage'] = round(
        (model_usage['total_queries'] / model_usage['total_queries'].sum()) * 100, 2
    )
    
    # Sort by total queries descending
    model_usage = model_usage.sort_values('total_queries', ascending=False).reset_index(drop=True)
    
    return model_usage

In [ ]:
print("TIMESTAMP column sample:")
print(df_agg['TIMESTAMP'].head())
print("\nTIMESTAMP column dtype:")
print(df_agg['TIMESTAMP'].dtype)


In [ ]:
df_copy = df_agg.copy()

df_copy['TIMESTAMP']
    
# Since TIMESTAMP is already datetime64[us], just extract the date
# df_copy['date'] = df_copy['TIMESTAMP'].dt.date

# daily_model_usage = df_copy.groupby(['date', 'SEMANTIC_MODEL_NAME']).agg({
#     'REQUEST_ID': 'count',  # Adjust column name if different
#     'USER_NAME': 'nunique'
# }).rename(columns={
#     'REQUEST_ID': 'queries_count',
#     'USER_NAME': 'unique_users'
# }).reset_index()

# # Sort by date and semantic model
# daily_model_usage = daily_model_usage.sort_values(
#     ['date', 'SEMANTIC_MODEL_NAME']
# ).reset_index(drop=True)
    


In [ ]:
# Get user activity by semantic model
user_activity_df = user_activity_by_semantic_model(df_agg)
print(user_activity_df)

# Get daily usage by semantic model
daily_usage_df = daily_usage_by_semantic_model(df_agg)
print(daily_usage_df)

# Get semantic model usage summary
model_summary_df = semantic_model_usage_summary(df_agg)
print(model_summary_df)

In [ ]:
user_activity_df

In [ ]:
def create_cortex_analyst_query_history(session, cortex_analyst_df, sf_intelligence_table="snowflake_intelligence.public.sf_intelligence_query_history"):
    """
    Creates joined query history by merging Snowflake Intelligence query history 
    with Cortex Analyst logs on generated SQL.
    
    Parameters
    ----------
    session : snowflake.snowpark.Session
        Active Snowpark session
    cortex_analyst_df : pandas.DataFrame
        DataFrame containing Cortex Analyst logs with GENERATED_SQL column
    sf_intelligence_table : str, optional
        Fully qualified name of the Snowflake Intelligence query history table
    
    Returns
    -------
    pandas.DataFrame
        Joined DataFrame with additional computed columns
    """
    
    # Load Snowflake Intelligence query history
    sfi_query_history = session.table(sf_intelligence_table)
    pd_sfi_query_history = sfi_query_history.to_pandas()
    
    # Set indexes for joining
    pd_sfi_query_history_indexed = pd_sfi_query_history.set_index("CLEANED_QUERY_TEXT")
    cortex_analyst_indexed = cortex_analyst_df.set_index("GENERATED_SQL")
    
    # Join on generated SQL (left index = cleaned_query_text, right index = generated_sql)
    ca_query_history = pd.merge(
        pd_sfi_query_history_indexed, 
        cortex_analyst_indexed, 
        left_index=True, 
        right_index=True,  # You can change to 'left', 'right', or 'outer' as needed
    )
    
    # Add computed columns
    ca_query_history['TOTAL_TIME'] = (
        ca_query_history["TOTAL_ELAPSED_TIME"] + ca_query_history["LATENCY_MS"]
    )
    
    ca_query_history['TOTAL_CREDITS_WH_AND_CA'] = (
        ca_query_history["CREDITS_ATTRIBUTED_COMPUTE"] + ca_query_history["CORTEX_ANALYST_CREDITS"]
    )
    
    # Reset index to make it a regular DataFrame
    ca_query_history = ca_query_history.reset_index()
    
    return ca_query_history

In [ ]:
glued_df = create_cortex_analyst_query_history(session, df_agg, "snowflake_intelligence.public.sf_intelligence_query_history")
glued_df

In [ ]:
# df_agg


sfi_query_history = session.table("snowflake_intelligence.public.sf_intelligence_query_history")
pd_sfi_query_history = sfi_query_history.to_pandas()
pd_sfi_query_history.set_index("CLEANED_QUERY_TEXT")
df_agg.set_index("GENERATED_SQL")
# pd_sfi_query_history
# ##joining on generated SQL
ca_query_history = pd.merge(pd_sfi_query_history, df_agg, left_index=True, right_index=True)
# ca_query_history['TOTAL_TIME'] = ca_query_history["TOTAL_ELAPSED_TIME"] + ca_query_history["LATENCY_MS"]
# ca_query_history['TOTAL_CREDITS_WH_AND_CA'] = ca_query_history["CREDITS_ATTRIBUTED_COMPUTE"] + ca_query_history["CORTEX_ANALYST_CREDITS"]

In [ ]:
ca_query_history['TOTAL_TIME'] = ca_query_history["TOTAL_ELAPSED_TIME"] + ca_query_history["LATENCY_MS"]
ca_query_history['TOTAL_CREDITS_WH_AND_CA'] = ca_query_history["CREDITS_ATTRIBUTED_COMPUTE"] + ca_query_history["CORTEX_ANALYST_CREDITS"]

total_cost_by_semantic_model = ca_query_history[['SEMANTIC_MODEL_NAME','TOTAL_CREDITS_WH_AND_CA']].groupby(['SEMANTIC_MODEL_NAME']).sum(['TOTAL_CREDITS_WH_AND_CA'])
total_cost_by_semantic_model